# About Evaluation, simple model(評価指標とシンプルモデル)

今回用いられている評価指標の"logloss"についての性質をまとめます（自己満）。<br>
加えて簡単なモデルを使って予測しようと思います。<br>
(去年作成したものを引用)


here, I think summarize the properties of the evaluation metric "logloss" and use a simple model to predict<br>
I quote the one I created last year.<br>
URL:https://www.kaggle.com/santohide/about-evaluation-simple-model-fors2<br>
Please advise me !!


## 1. 評価指標のグラフ作成 (Creating graphs of evaluation indicators)
今回用いられる評価指標は"logloss"と呼ばれるもので式は以下(The equation called logloss is as follows)：
$$
Logloss = - \frac{1}{n}\sum_{i=1}^{n}[y_i\log({\hat y_i})+(1-y_i)\log((1-{\hat y_i}))]
$$

where

・ n is the number of games played<br>
・ $ŷ$ i is the predicted probability of team 1 beating team 2<br>
・ $y_i$ is 1 if team 1 wins, 0 if team 2 wins<br>
・ log is the natural logarithm<br>

$n=1$とした時の$ŷ$と$y_i$を用いた評価指標の差がどれだけになるのかをグラフを用いて表す

Let's use a graph to show how much the difference between the evaluation indices using $ŷ$ and $y_i$ will be when $n=1$.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

In [ ]:
#result y_i =0 or 1
fig,axes =plt.subplots(1,2,figsize=(16,5))
graph_title = ["evaluation score(y is 0)","evaluation score(y is 1)"]
for i in range(0,2):
    y = np.tile(i,98)
    y_hat = np.linspace(0.01,0.99,98)
    log_loss = -(y*np.log(y_hat)+ (1-y)*(np.log(1-y_hat)))
    axes[i].set_title(graph_title[i])
    axes[i].plot(y_hat,log_loss)
#縦軸が評価指標のスコア、横軸が自分が予測したyの値

* もしも結果が1($y=1$)で予測が0(${\hat y}=0$)の場合は値がとんでもないことになる(結果と予想が逆の場合も同様です。)<br>If the result is 1 ($y=1$) and the prediction is 0 (${\hat y}=0$), The same is true if the result and prediction are reversed.

* 全部0.5と予測すれば評価指標は1を下回る(大体0.7くらい)<br>If we predict 0.5, the evaluation index will be below 1 (about 0.7).

## まとめ1 conclusion1

* 1st stageの予測で0.7超えたらそのモデルはちょっとおかしいかも<br>If the 1st stage prediction exceeds 0.7, that model may have a problem
* しっかりデータを抽出してロジスティック回帰を行えばそこそこいい予測は出るのではないかと推測しています<br>I'm guessing that if you extract the data properly and do a logistic regression, you can get a good prediction.

## 2.モデル作成 model establishing
その年のシーズンの勝率を説明変数としてロジスティック回帰を用いて予測することを考えていきます<br>
データの加工等は以下のnotebookを真似しました<br>
ありがとうございます！！

https://www.kaggle.com/theoviel/ncaa-starter-the-simpler-the-better

To predict, I consider using logistic model with the season's winning percentage as explanatory variable<br>
Data processing and more was based on the above NOTEBOOK.
Thanks!!

In [ ]:
import os
import re
import sklearn
from sklearn.linear_model import *
import seaborn as sns

In [ ]:
DATA_PATH = '../input/mens-march-mania-2022/MDataFiles_Stage1/'
df_seeds = pd.read_csv(DATA_PATH + "MNCAATourneySeeds.csv")
df_season_results = pd.read_csv(DATA_PATH + "MRegularSeasonCompactResults.csv")
df_season_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

In [ ]:
#各シーズン,チームの勝利数
num_win = df_season_results.groupby(['Season', 'WTeamID']).count()
num_win = num_win.reset_index()[['Season', 'WTeamID', 'DayNum']].rename(columns={"DayNum": "NumWins", "WTeamID": "TeamID"})
#各シーズン,チームの敗北数
num_loss = df_season_results.groupby(['Season', 'LTeamID']).count()
num_loss = num_loss.reset_index()[['Season', 'LTeamID', 'DayNum']].rename(columns={"DayNum": "NumLosses", "LTeamID": "TeamID"})

In [ ]:
df_features_season_w = df_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
df_features_season_l = df_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})
df_features_season = pd.concat([df_features_season_w, df_features_season_l], 0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)
df_features_season = df_features_season.merge(num_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(num_loss, on=['Season', 'TeamID'], how='left')
df_features_season.fillna(0, inplace=True)
df_features_season['WinRatio'] = df_features_season['NumWins'] / (df_features_season['NumWins'] + df_features_season['NumLosses'])
df_features_season.isnull().sum()

In [ ]:
def get_round(day):
    round_dic = {134: 0, 135: 0, 136: 1, 137: 1, 138: 2, 139: 2, 143: 3, 144: 3, 145: 4, 146: 4, 152: 5, 154: 6}
    try:
        return round_dic[day]
    except:
        print(f'Unknow day : {day}')
        return 0
def add_loosing_matches(win_df):
    win_rename = {
        "WTeamID": "TeamIdA", 
        "WScore" : "ScoreA", 
        "LTeamID" : "TeamIdB",
        "LScore": "ScoreB",
        'WinRatioW' : 'WinRatioA',
        'WinRatioL' : 'WinRatioB',
     }
    
    lose_rename = {
        "WTeamID": "TeamIdB", 
        "WScore" : "ScoreB", 
        "LTeamID" : "TeamIdA",
        "LScore": "ScoreA",
        'WinRatioW' : 'WinRatioB',
        'WinRatioL' : 'WinRatioA',
    }
    
    win_df = win_df.copy()
    lose_df = win_df.copy()
    
    win_df = win_df.rename(columns=win_rename)
    lose_df = lose_df.rename(columns=lose_rename)
    
    return pd.concat([win_df, lose_df], 0, sort=False)

In [ ]:
df_tourney_results = pd.read_csv(DATA_PATH + "MNCAATourneyCompactResults.csv")
df_tourney_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)
df_tourney_results
df_tourney_results['Round'] = df_tourney_results['DayNum'].apply(get_round)

In [ ]:
df = df_tourney_results.copy()
df = df[df['Season'] >= 2003].reset_index(drop=True)

df.head()
df = pd.merge(
    df, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'WTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1)
df = pd.merge(
    df, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'LTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1)
df = df.drop(['Seed_x','Seed_y'], axis=1)
df.isnull().sum()

In [ ]:
df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsW',
    'NumLosses': 'NumLossesW',
    'WinRatio': 'WinRatioW',
}).drop(columns='TeamID', axis=1)
df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'LTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsL',
    'NumLosses': 'NumLossesL',
    'WinRatio': 'WinRatioL',
}).drop(columns='TeamID', axis=1)
df = add_loosing_matches(df)
df = df.drop(["NumWinsW","NumLossesW","NumWinsL","NumLossesL"],axis=1)

df['WinRatioDiff'] = df['WinRatioA'] - df['WinRatioB']
df['ScoreDiff'] = df['ScoreA'] - df['ScoreB']
df['WinA'] = (df['ScoreDiff'] > 0).astype(int)

In [ ]:
df_test = pd.read_csv(DATA_PATH + "MSampleSubmissionStage1.csv")
df_test['Season'] = df_test['ID'].apply(lambda x: int(x.split('_')[0]))
df_test['TeamIdA'] = df_test['ID'].apply(lambda x: int(x.split('_')[1]))
df_test['TeamIdB'] = df_test['ID'].apply(lambda x: int(x.split('_')[2]))
df_test = pd.merge(
    df_test,
    df_seeds,
    how='left',
    left_on=['Season', 'TeamIdA'],
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1)
df_test = pd.merge(
    df_test, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'TeamIdB'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1)
df_test = df_test.drop(['Seed_x','Seed_y'], axis=1)

In [ ]:
df_test = pd.merge(
    df_test,
    df_features_season,
    how='left',
    left_on=['Season', 'TeamIdA'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsA',
    'NumLosses': 'NumLossesA',
    'WinRatio': 'WinRatioA',
}).drop(columns='TeamID', axis=1)
df_test = pd.merge(
    df_test,
    df_features_season,
    how='left',
    left_on=['Season', 'TeamIdB'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsB',
    'NumLosses': 'NumLossesB',
    'WinRatio': 'WinRatioB',
}).drop(columns='TeamID', axis=1)
df_test=df_test.drop(["NumWinsA","NumLossesA","NumWinsB","NumLossesB"],axis=1)
df_test['WinRatioDiff'] = df_test['WinRatioA'] - df_test['WinRatioB']
df_test

## モデル作成 Model creation

シーズンの勝率を説明変数としたロジスティックモデルの作成をしました<br>
I created a logistic model with the season's winning percentage as the explanatory variable.


* 2015-2019, 2021のデータを使用 Use data from 2015-2019 and 2021 
* 結果をその年のシーズンデータから予測 Predict results based on the year's season data.

In [ ]:
features = [
    'WinRatioA',
    'WinRatioB',
    'WinRatioDiff',
]

def logistic_reg(df, df_test_=None, plot=False, verbose=0):
    seasons = df['Season'].unique()
    cvs = []
    pred_tests = []
    target = "WinA"
    
    seasons_for_pred = [2016,2017,2018,2019,2021]

    for season in seasons_for_pred:
        print(f'\nValidating on season {season}')
        
        df_train = df[df['Season'] == season].reset_index(drop=True).copy()
        df_test = df_test_[df_test_["Season"]==season].copy()        
        
        model = LogisticRegression(C=10)
        model.fit(df_train[features], df_train[target])
        if df_test is not None:
            pred_test = model.predict_proba(df_test[features])[:, 1]               
            pred_tests.append(pred_test)
            print(pred_test.shape)
        
    return np.array(pred_tests).reshape(2278*5,1)
pred_test = logistic_reg(df, df_test, plot=False,verbose=0)

In [ ]:
pred_test.shape

In [ ]:
df_test[['ID', 'Pred']].copy()

In [ ]:
sub = df_test[['ID', 'Pred']].copy()
sub['Pred'] = pred_test
sub.to_csv('submission.csv', index=False)

In [ ]:
_ = sns.histplot(sub['Pred'])

## まとめ2 Conclusion2

* このモデルの結果は0.64くらい The result for this model is about 0.64
* ここから自分なりの工夫を凝らしてモデルを作成すれば、良い結果が出ると思います！ 
* If we can improve our model, I think we'll see some good results!

# まとめ

雑なノートブックで申し訳ありません
向上心を持ってデータ分析に励んでください!

Be ambitious and work hard on your data analysis!